In [1]:
import pandas as pd
import datetime
import mariadb
import numpy as np
from pathlib import Path

In [2]:
csv_path = Path().cwd().parent.joinpath('voter_history_files/3-1-2016 - MARCH 1, 2016 PPP:SPECIAL ELECTION.csv')
import_csv = pd.read_csv(csv_path, converters={'County Name': str, 'Voter Registration Number': str, 'Election Date':str, 'Election Type':str, 'Party': str, 'Ballot Style':str, 'Absentee':str, 'Provisional':str, 'Supplemental':str})
output_name = 'primary_2016.csv'

In [3]:
import_csv

,County Name,Voter Registration Number,Election Date,Election Type,Party,Ballot Style,Absentee,Provisional,Supplemental
0,APPLING,00004137,03/01/2016,PPP,REPUBLICAN,REGULAR,N,N,N
1,APPLING,00043572,03/01/2016,PPP,REPUBLICAN,REGULAR,N,N,N
2,APPLING,00045353,03/01/2016,PPP,DEMOCRAT,REGULAR,N,N,N
3,APPLING,00047002,03/01/2016,PPP,REPUBLICAN,REGULAR,N,N,N
4,APPLING,00069178,03/01/2016,PPP,REPUBLICAN,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...
2066813,BRYAN,03470524,03/01/2016,PPP,DEMOCRAT,REGULAR,N,N,N
2066814,FULTON,05323544,03/01/2016,PPP,DEMOCRAT,REGULAR,N,N,N
2066815,UPSON,04069157,03/01/2016,PPP,DEMOCRAT,MAIL IN,Y,N,N
2066816,BIBB,00183718,03/01/2016,PPP,DEMOCRAT,REGULAR,N,N,N


In [4]:
df = import_csv.copy()

In [5]:
df = df.rename(columns={'County Name': 'county_name', 'Voter Registration Number': 'voter_registration_number', 'Election Date': 'election_date', 'Election Type': 'election_type', 'Party': 'party', 'Ballot Style': 'ballot_style', 'Absentee': 'absentee', 'Provisional': 'provisional', 'Supplemental':'supplemental'})

In [6]:
def add_row_names(column):
    counter = 0
    row_name_list = []
    for row in column:
        counter += 1
        row_name_list.append(counter)
    return row_name_list

In [7]:
df.insert(0, 'row_names', add_row_names(df['county_name']))

In [8]:
def column_float_to_strings(column):
    string_list = []
    for entry in column:
        if str(entry) == 'nan':
            string_list.append("nan")
        else:
            string_list.append(str(entry))
    return string_list

In [9]:
def remove_last_two_digits(column):
    string_list = []
    for entry in column:
        string_list.append(entry[0:-2])
    return string_list

In [10]:
df['voter_registration_number'][df['voter_registration_number'].isnull()]

Series([], Name: voter_registration_number, dtype: object)

In [11]:
df

,row_names,county_name,voter_registration_number,election_date,election_type,party,ballot_style,absentee,provisional,supplemental
0,1,APPLING,00004137,03/01/2016,PPP,REPUBLICAN,REGULAR,N,N,N
1,2,APPLING,00043572,03/01/2016,PPP,REPUBLICAN,REGULAR,N,N,N
2,3,APPLING,00045353,03/01/2016,PPP,DEMOCRAT,REGULAR,N,N,N
3,4,APPLING,00047002,03/01/2016,PPP,REPUBLICAN,REGULAR,N,N,N
4,5,APPLING,00069178,03/01/2016,PPP,REPUBLICAN,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...,...
2066813,2066814,BRYAN,03470524,03/01/2016,PPP,DEMOCRAT,REGULAR,N,N,N
2066814,2066815,FULTON,05323544,03/01/2016,PPP,DEMOCRAT,REGULAR,N,N,N
2066815,2066816,UPSON,04069157,03/01/2016,PPP,DEMOCRAT,MAIL IN,Y,N,N
2066816,2066817,BIBB,00183718,03/01/2016,PPP,DEMOCRAT,REGULAR,N,N,N


In [12]:
def reformat_date(column):
    date_list = []
    for entry in column:
        date_list.append(f"{entry[6:10]}" + "-" + f"{entry[0:2]}" + "-" + f"{entry[3:5]}")
    return date_list

In [13]:
df['election_date'] = reformat_date(df['election_date'])

In [14]:
df

,row_names,county_name,voter_registration_number,election_date,election_type,party,ballot_style,absentee,provisional,supplemental
0,1,APPLING,00004137,2016-03-01,PPP,REPUBLICAN,REGULAR,N,N,N
1,2,APPLING,00043572,2016-03-01,PPP,REPUBLICAN,REGULAR,N,N,N
2,3,APPLING,00045353,2016-03-01,PPP,DEMOCRAT,REGULAR,N,N,N
3,4,APPLING,00047002,2016-03-01,PPP,REPUBLICAN,REGULAR,N,N,N
4,5,APPLING,00069178,2016-03-01,PPP,REPUBLICAN,REGULAR,N,N,N
...,...,...,...,...,...,...,...,...,...,...
2066813,2066814,BRYAN,03470524,2016-03-01,PPP,DEMOCRAT,REGULAR,N,N,N
2066814,2066815,FULTON,05323544,2016-03-01,PPP,DEMOCRAT,REGULAR,N,N,N
2066815,2066816,UPSON,04069157,2016-03-01,PPP,DEMOCRAT,MAIL IN,Y,N,N
2066816,2066817,BIBB,00183718,2016-03-01,PPP,DEMOCRAT,REGULAR,N,N,N


In [18]:
df.value_counts('election_type')

election_type
PPP    2066818
Name: count, dtype: int64

In [15]:
output_path = Path().cwd().parent.joinpath('output_csv/primary_2016.csv')
df.to_csv(output_path)